In [1]:
import numpy as np

import scipy as sp
from scipy import spatial

import matplotlib.pyplot as plt # side-stepping mpl backend

from nltk.stem.lancaster import LancasterStemmer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import pairwise
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import cross_validation
import heapq
import string
import re

from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter

vectorizer = CountVectorizer()
st = LancasterStemmer()

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
with open("TestData/train.dat", "r") as fh:
    #with open("TestData/Test/training_out.dat", "r") as fh:
    linesOfTrainData = fh.readlines()
print(len(linesOfTrainData))

with open("TestData/format.dat", "r") as fh:
#with open("TestData/Test/format_out.dat", "r") as fh:
    linesOfFormat = fh.readlines()
print(len(linesOfFormat))

with open("TestData/test.dat", "r") as fh:
#with open("TestData/Test/test_out.dat", "r") as fh:
    linesOfTestData = fh.readlines()
print(len(linesOfTestData))


18506
18506
18506


In [4]:
##stopwords and stemming for Training

print ("First line before cleanup from Training Data: ",linesOfTrainData[0])

stops = set(stopwords.words('english'))

linesOfTrainDataAfterPreProcessing=[]
for line in linesOfTrainData:
    newLine = []
    for w in line.split():
        if w.lower()not in stops:
            newLine.append(w.lower())     #for stopwords
            finalLine = " ".join(newLine)
            finalLine = finalLine.replace(w, st.stem(w)) #for stemming
            finalLine = finalLine.translate(str.maketrans('','',string.punctuation)) #for punctuation
            finalLine = re.sub("\d+", " ", finalLine)# for numbers
    linesOfTrainDataAfterPreProcessing.append(finalLine)


linesOfTrainData = linesOfTrainDataAfterPreProcessing

print ("\n\nFirst line after cleanup from Training Data: ",linesOfTrainData[0])

First line before cleanup from Training Data:  +1	This book is such a life saver.  It has been so helpful to be able to go back to track trends, answer pediatrician questions, or communicate with each other when you are up at different times of the night with a newborn.  I think it is one of those things that everyone should be required to have before they leave the hospital.  We went through all the pages of the newborn version, then moved to the infant version, and will finish up the second infant book (third total) right as our baby turns 1.  See other things that are must haves for baby at [...]



First line after cleanup from Training Data:    book life saver helpful able go back track trends answer pediatrician questions communicate different times night newborn think one things everyone required leave hospital went pages newborn version moved infant version finish second infant book third total right baby turns   see things must haves baby 


In [5]:
##stopwords and stemming for Training

print ("First line before cleanup from Testing Data: ",linesOfTestData[0])

stops = set(stopwords.words('english'))

linesOfTestDataAfterPreProcessing=[]
for line in linesOfTestData:
    newLine = []
    for w in line.split():
        if w.lower()not in stops:
            newLine.append(w.lower())     #for stopwords
            finalLine = " ".join(newLine)
            finalLine = finalLine.replace(w, st.stem(w)) #for stemming
            finalLine = finalLine.translate(str.maketrans('','',string.punctuation)) #for punctuation
            finalLine = re.sub("\d+", " ", finalLine)# for numbers
    linesOfTestDataAfterPreProcessing.append(finalLine)


linesOfTestData = linesOfTestDataAfterPreProcessing

print ("\n\nFirst line after cleanup from Testing Data: ",linesOfTestData[0])

First line before cleanup from Testing Data:  Perfect for new parents. We were able to keep track of baby's feeding, sleep and diaper change schedule for the first two and a half months of her life. Made life easier when the doctor would ask questions about habits because we had it all right there!



First line after cleanup from Testing Data:  perfect new parents able keep track babys feeding sleep diaper change schedule first two half months life made life easier doctor would ask questions habits right there


In [6]:
# get a frequency count for all words in the Training docs



wordsInTrainingSet = set()

wordsCountInTrainingSet = Counter()
for d in linesOfTrainData:
    for w in d.split():
        if w == "+1" or w == "-1":
            continue
        else:
            if w not in wordsInTrainingSet:
                wordsInTrainingSet.add(w)
                wordsCountInTrainingSet[w]=1
            else:
                wordsCountInTrainingSet[w] += 1
print("Number of unique words: %d." % len(wordsInTrainingSet))




Number of unique words: 36405.


In [7]:
#Top percent of training data 
top_percentile = 0.2

topPercentileOfTraining = wordsCountInTrainingSet.most_common(int(round(len(wordsCountInTrainingSet)*top_percentile)))

wordsInTrainingSet2 = set()
for word in topPercentileOfTraining:
    wordsInTrainingSet2.add(word[0])
    
print(len(wordsInTrainingSet2))


7281


In [9]:
# get a frequency count for all words in the Test docs
wordsInTestSet  = set()

wordsCountInTestingSet = Counter()

for d in linesOfTestData:
    for w in d.split():
        if w not in wordsInTestSet:
            wordsInTestSet.add(w)
            wordsCountInTestingSet[w]=1
        else:
            wordsCountInTestingSet[w] += 1
print("Number of unique words: %d." % len(wordsInTestSet))






Number of unique words: 36355.


In [10]:
#Top percent of testing data 
top_percentile = 0.2

topPercentileOfTesting = wordsCountInTestingSet.most_common(int(round(len(wordsCountInTestingSet)*top_percentile)))

wordsInTestingSet2 = set()
for word in topPercentileOfTesting:
    wordsInTestingSet2.add(word[0])
    
print(len(wordsInTestingSet2))

7271


In [11]:
wordsInSet = set()
wordsList= []

wordsInSet =  set.intersection(wordsInTrainingSet2,wordsInTestingSet2)
wordsList = list(wordsInSet)

print("Number of unique words: %d." % len(wordsInSet))

Number of unique words: 6357.


In [12]:
from collections import Counter
c = Counter(wordsInSet)

top_percentile = 1.0 # 0.2 Precentile
features = c.most_common(int(round(len(c)*top_percentile)))
features_counts = len(features)

print(type(features))
print(features_counts)


<class 'list'>
6357


In [13]:
#print(features)
print(features[:1][0])
print(features[:1][0][1])

featureList= []


for feature in features:
    featureList.append(feature[0])
    

('installed', 1)
1


In [14]:
# #Buid matrix

from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    dim = len(featureList)
    feature_set = set(featureList[:dim])
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        set_d = set(d)
        
        d = list(set.intersection(feature_set,set_d))
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        set_d = set(d)
        
        d = list(set.intersection(feature_set,set_d))
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [16]:
#print(linesOfTrainData[:1])
docslinesOfTrainData = [l.split() for l in linesOfTrainData]
matOflinesOfTrainData  = build_matrix(docslinesOfTrainData)
csr_info(matOflinesOfTrainData)

#print (matOflinesOfTrainData[:1])

#print(linesOfTestData[:1])
docslinesOfTestData = [l.split() for l in linesOfTestData]
matOflinesOfTestData  = build_matrix(docslinesOfTestData)
csr_info(matOflinesOfTestData)
#print (matOflinesOfTestData[:1])



 [nrows 18506, ncols 6357, nnz 741757]
 [nrows 18506, ncols 6357, nnz 739621]


In [17]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat


In [18]:
mat2linesOfTrainData = csr_idf(matOflinesOfTrainData, copy=True)
mat3linesOfTrainData = csr_l2normalize(mat2linesOfTrainData, copy=True)
print("mat1:", matOflinesOfTrainData[15,:20].todense(), "\n")
print("mat2:", mat2linesOfTrainData[15,:20].todense(), "\n")
print("mat3:", mat3linesOfTrainData[15,:20].todense())

mat1: [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]] 

mat2: [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]] 

mat3: [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]


In [19]:

mat2linesOfTestData = csr_idf(matOflinesOfTestData, copy=True)
mat3linesOfTestData = csr_l2normalize(mat2linesOfTestData, copy=True)
print("mat1:", matOflinesOfTestData[15,:20].todense(), "\n")
print("mat2:", mat2linesOfTestData[15,:20].todense(), "\n")
print("mat3:", mat3linesOfTestData[15,:20].todense())



mat1: [[ 0.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.
   1.  0.]] 

mat2: [[ 0.          1.31025837  2.71026816  0.          0.          3.75050425
   0.          0.          0.          0.          0.          0.          0.
   3.01681098  0.          7.11780008  2.34092     0.          2.46193678
   0.        ]] 

mat3: [[ 0.          0.06509744  0.134654    0.          0.          0.18633595
   0.          0.          0.          0.          0.          0.          0.
   0.14988394  0.          0.35363299  0.11630371  0.          0.12231617
   0.        ]]


In [20]:
linesOfTrainData_Transformed = mat3linesOfTrainData
linesOfTestData_Transformed = mat3linesOfTestData

In [22]:
print(docslinesOfTrainData[:1]) 
print('-------')
print(linesOfTrainData_Transformed[:1])

[['book', 'life', 'saver', 'helpful', 'able', 'go', 'back', 'track', 'trends', 'answer', 'pediatrician', 'questions', 'communicate', 'different', 'times', 'night', 'newborn', 'think', 'one', 'things', 'everyone', 'required', 'leave', 'hospital', 'went', 'pages', 'newborn', 'version', 'moved', 'infant', 'version', 'finish', 'second', 'infant', 'book', 'third', 'total', 'right', 'baby', 'turns', 'see', 'things', 'must', 'haves', 'baby']]
-------
  (0, 0)	0.128410879293
  (0, 1)	0.174238594491
  (0, 2)	0.129461452738
  (0, 3)	0.108895242227
  (0, 4)	0.13384089994
  (0, 5)	0.117964958141
  (0, 6)	0.115133026601
  (0, 7)	0.106124899303
  (0, 8)	0.169508361842
  (0, 9)	0.259049892766
  (0, 10)	0.0984793818715
  (0, 11)	0.237913189412
  (0, 12)	0.220694599699
  (0, 13)	0.251089155349
  (0, 14)	0.158443308718
  (0, 15)	0.0463984397886
  (0, 16)	0.13537179031
  (0, 17)	0.0477038364187
  (0, 18)	0.211287506083
  (0, 19)	0.279807488023
  (0, 20)	0.0948526255239
  (0, 21)	0.0940413364635
  (0, 22)

In [31]:

print(docslinesOfTestData[:1])
print('-------')
print(linesOfTestData_Transformed[:1])

[['perfect', 'new', 'parents', 'able', 'keep', 'track', 'babys', 'feeding', 'sleep', 'diaper', 'change', 'schedule', 'first', 'two', 'half', 'months', 'life', 'made', 'life', 'easier', 'doctor', 'would', 'ask', 'questions', 'habits', 'right', 'there']]
-------
  (0, 0)	0.161643018545
  (0, 1)	0.121366821868
  (0, 2)	0.132385703388
  (0, 3)	0.203971980036
  (0, 4)	0.118451939549
  (0, 5)	0.269643545927
  (0, 6)	0.0679313665398
  (0, 7)	0.312069482433
  (0, 8)	0.190435644151
  (0, 9)	0.143417487951
  (0, 10)	0.193502992091
  (0, 11)	0.101951465531
  (0, 12)	0.144135088049
  (0, 13)	0.14051596494
  (0, 14)	0.300731370365
  (0, 15)	0.354112791525
  (0, 16)	0.208927734609
  (0, 17)	0.369027892788
  (0, 18)	0.142709684748
  (0, 19)	0.099374262205
  (0, 20)	0.127641031244
  (0, 21)	0.194447816041
  (0, 22)	0.182004208371
  (0, 23)	0.156408916367
  (0, 24)	0.168756317623


In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:

def CalculateCosine(vt,vs):
        cosineSimilarityValue = cosine_similarity(vt,vs)
        return cosineSimilarityValue


In [ ]:
cosineSimilarityValue = CalculateCosine(linesOfTestData_Transformed,linesOfTrainData_Transformed)


In [28]:
print(len(cosineSimilarityValue))

18506


In [31]:

f = open('TestData/Test/format.dat', 'w')
count = 0
for row in cosineSimilarityValue:

    k=72
    partitioned_row_byindex = np.argpartition(-row, k)  
    similar_index = partitioned_row_byindex[:k]
    
    neighbourReviewTypeList = []
    neighbourReviewTypeNegative = 0
    neighbourReviewTypePositive = 0

    for index in similar_index:

        if linesOfTrainData[index].strip()[0] == '-':
            neighbourReviewTypeNegative+=1
        elif linesOfTrainData[index].strip()[0] == '+':
            neighbourReviewTypePositive+=1
            
    
    if neighbourReviewTypeNegative > neighbourReviewTypePositive:
        f.write('-1\n')
        count+=1
    else:
        f.write('+1\n')
        count+=1

print("count : ",count)
print("--The End--")

count :  18506
--The End--


In [32]:
with open("TestData/format.dat", "r") as fh:
#with open("TestData/Test/format_out.dat", "r") as fh:
    linesOfFormat = fh.readlines()
print(len(linesOfFormat))


18506
